# Setup

In [7]:
# Load pandas and numpy
import pandas as pd
import numpy as np

# For text preprocessing
import spacy
nlp = spacy.load('en_core_web_sm')


# For loading Data and models
import joblib
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


# For evaluation we will use accuracy, f1-score, precesion, recall and confusion matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix


In [8]:
filepath_test = '/Users/thebekhruz/Desktop/nlu/EvidenceExplorer/data/validate/dev.csv'
df_test = pd.read_csv(filepath_test)
df_test.head()


,Claim,Evidence,label
0,We should legalize the growing of coca leaf,"Seeing the involvement of the coca growers, th...",0
1,We should limit the use of birth control,"Although FDA-approved for contraceptive use, S...",0
2,We should prohibit flag burning,The case of Texas v. Johnson was appealed to t...,0
3,The vow of celibacy should be abandoned,Much of the encyclical is spent discussing rea...,1
4,We should further exploit natural gas,Helium is typically produced by separating it ...,0


In [9]:
# Load the models
try:
    clf = joblib.load('logistic_regression_model.pkl')
    clf_rf = joblib.load('random_forest_model.pkl')
    clf_gb = joblib.load('gradient_boosting_model.pkl')
    model = Doc2Vec.load('doc2vec_model')
    print('Models are loaded')
except:
    print('Error: models are not loaded')


Models are loaded


# Majority Voting

In [10]:
# Implementing the majority voting function with 3 classifiers
def majority_voting(clf1, clf2, clf3, X):
    y_pred1 = clf1.predict(X)
    y_pred2 = clf2.predict(X)
    y_pred3 = clf3.predict(X)
    y_pred = []
    for i in range(len(y_pred1)):
        if y_pred1[i] + y_pred2[i] + y_pred3[i] > 1:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return np.array(y_pred)


# Infer the vectors for the training data
def infer_vectors(model, tagged_documents):
    vectors = [model.infer_vector(doc.words) for doc in tagged_documents]
    return np.array(vectors)

def create_tagged_document(df):
    tagged_data = []
    for i, text in enumerate(df['Claim'] + ' ' + df['Evidence']):
        # Process the text with the spaCy language model
        doc = nlp(text)
        # Tokenize and lemmatize the text, removing stopwords
        tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
        # Create a TaggedDocument for each row in the dataframe
        tagged_data.append(TaggedDocument(words=tokens, tags=[str(i)]))  # Tags are typically strings
    return tagged_data


In [11]:
tagged_data_test = create_tagged_document(df_test)
vectors_test = infer_vectors(model, tagged_data_test)
y_pred = majority_voting(clf, clf_rf, clf_gb, vectors_test)

# Takes around 50 seconds

In [12]:
# Calculate the accuracy
accuracy = accuracy_score(df_test['label'], y_pred)
precision = precision_score(df_test['label'], y_pred)
recall = recall_score(df_test['label'], y_pred)
f1 = f1_score(df_test['label'], y_pred)
conf_matrix = confusion_matrix(df_test['label'], y_pred)


print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Confusion Matrix: \n{conf_matrix}')


Accuracy: 0.743334458319271
Precision: 0.6681034482758621
Recall: 0.0969355847404628
F1 Score: 0.16930638995084654
Confusion Matrix: 
[[4250   77]
 [1444  155]]
